# 08 - Strategy Optimizer

## Overview
Enumerate and rank pit stop strategies.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat, optimizer

## Load

In [2]:
# Example optimization
current_state = {
    'current_lap': 20,
    'total_laps': 57,
    'compounds_available': ['SOFT', 'MEDIUM', 'HARD'],
    'base_lap_time_s': 90.0,
    'deg_rate_ms_per_lap': 50,
    'pit_loss_s': 24.0,
}

## Transform: Optimize

In [3]:
strategies = optimizer.optimize_strategy(current_state, max_stops=2)
print(strategies.head(10))

Evaluating 141 candidate strategies...
    n_stops stop_laps             compounds  exp_finish_time_s  \
0         0        []              ["SOFT"]           3364.225   
1         0        []            ["MEDIUM"]           3364.225   
2         0        []              ["HARD"]           3364.225   
15        1      [37]      ["SOFT", "SOFT"]           3371.225   
19        1      [40]  ["MEDIUM", "MEDIUM"]           3371.225   
18        1      [40]      ["SOFT", "SOFT"]           3371.225   
17        1      [37]      ["HARD", "HARD"]           3371.225   
16        1      [37]  ["MEDIUM", "MEDIUM"]           3371.225   
20        1      [40]      ["HARD", "HARD"]           3371.225   
21        1      [43]      ["SOFT", "SOFT"]           3372.125   

                                        strategy_json  delta_to_best_s  
0   {"n_stops": 0, "stop_laps": [], "compounds": [...              0.0  
1   {"n_stops": 0, "stop_laps": [], "compounds": [...              0.0  
2   {"n_stops":

## Save

In [4]:
features_dir = config.paths()['data_features']
io_flat.write_parquet(strategies, features_dir / 'strategy_decisions.parquet')
print('✓ Saved strategy decisions')

✓ Saved strategy_decisions.parquet: 10 rows, 6 cols
✓ Saved strategy decisions


## Repro Notes

- Enumerated pit stop strategies
- Simulated expected finish times
- Ranked by estimated performance

# New: Risk-Aware Optimization (Monte Carlo, CVaR, Win Probability)

Use degradation quantile models and calibrated hazard probabilities to compute risk metrics: P50/P90 time, CVaR, P(win vs target).

In [ ]:
# Run risk-aware optimization
from f1ts import optimizer, io_flat, config
import pandas as pd

# Load models
qm = io_flat.load_model(config.paths()['models'] / 'degradation_quantiles.pkl') if (config.paths()['models'] / 'degradation_quantiles.pkl').exists() else None
haz_model = io_flat.load_model(config.paths()['models'] / 'hazard_model.pkl') if (config.paths()['models'] / 'hazard_model.pkl').exists() else None
calibrator = io_flat.load_model(config.paths()['models'] / 'hazard_calibrator.pkl') if (config.paths()['models'] / 'hazard_calibrator.pkl').exists() else None

models = {'quantile_models': qm, 'hazard_model': haz_model, 'hazard_calibrator': calibrator}

# Define current race state (example; replace with actual selected race context)
current_state = {
    'session_key': '2023_1_R',
    'total_laps': 57,
    'allowed_compounds': ['SOFT', 'MEDIUM', 'HARD'],
}

ranked = optimizer.optimize_strategy(
    current_state=current_state,
    max_stops=2,
    risk_aware=True,
    models=models,
    target_time=None  # set a rival time to compute win probability
)

ranked.head(10)